In [59]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import pickle


In [60]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [61]:
#preprocessing
data=data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [62]:
# encode categorical data
label_encoder_gender = LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [63]:
data['Gender'].value_counts()


Gender
1    5457
0    4543
Name: count, dtype: int64

In [64]:
#one hot encode geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(
    data[['Geography']]
).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [65]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [66]:
geo_encoded_df = pd.DataFrame(
    geo_encoder,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [67]:
## combine the encoded geography data with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [68]:
#split data into features and target
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']


In [69]:

#split the data into training and testing sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

##Scale the data
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [70]:
## save the encoderand scaler for later use
with open('label_encoder_gender.pkl', 'wb') as filter:
    pickle.dump(label_encoder_gender, filter)
with open('onehot_encoder_geo.pkl', 'wb') as filter:
    pickle.dump(onehot_encoder_geo, filter)
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

TRAIN ANN REGRESSION

In [71]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [72]:
#Build the ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),##HL1 connected with input layer
    Dense(32,activation='relu'),##HL2       
    Dense(1)##output layer(linear activation for regression)

    
])

In [73]:
model.compile(optimizer='adam',loss='mae',metrics=['mae'])

In [74]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 64)                832       
                                                                 
 dense_16 (Dense)            (None, 32)                2080      
                                                                 
 dense_17 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [75]:
##set up tensorboard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [76]:
##set up early stopping 
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [77]:
#training the model
history=model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[early_stopping_callback,tensorflow_callback])

Epoch 1/100
250/250 [==============================] - 3s 5ms/step - loss: 100376.3516 - mae: 100376.3516 - val_loss: 98508.5156 - val_mae: 98508.5156
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 99578.0391 - mae: 99578.0391 - val_loss: 96874.1953 - val_mae: 96874.1953
Epoch 3/100
250/250 [==============================] - 1s 6ms/step - loss: 96727.6719 - mae: 96727.6719 - val_loss: 92673.5156 - val_mae: 92673.5156
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 91165.5469 - mae: 91165.5469 - val_loss: 85756.1016 - val_mae: 85756.1016
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 83128.1250 - mae: 83128.1250 - val_loss: 76888.7031 - val_mae: 76888.7031
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 73769.1562 - mae: 73769.1562 - val_loss: 67772.4297 - val_mae: 67772.4297
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 64834.7617 - mae: 64834.7617 

In [78]:
## load tensorboard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [79]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 11256), started 0:06:33 ago. (Use '!kill 11256' to kill it.)

In [81]:
##evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 [==============================] - 1s 11ms/step - loss: 50280.5312 - mae: 50280.5312
Test MAE: 50280.53125


In [82]:
model.save('regression_model.h5')

c:\Users\dell\Documents\GitHub\ANN-Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
